<a href="https://colab.research.google.com/github/Wency-WangC/Nice-Little-Tools/blob/main/converter_v2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
### Lunar Date Converter with Dual Input Modes for Colab
# 安装依赖
!pip install -q opencc-python-reimplemented pandas openpyxl ipywidgets

from google.colab import files
import pandas as pd
import io, os
from opencc import OpenCC
import re
import ipywidgets as widgets
from IPython.display import display, clear_output

# 简体转繁体引擎
cc = OpenCC('s2t')

# 全局存储列表
entries = []

# 校验函数

def validate_entry(rec):
    """
    按照给定规则校验单条记录，返回错误信息或 None
    规则：
    1. 日存在时，必须同时输入月和年；月存在时，必须输入年。
    2. 如果没有年号，则必须同时填写国号和帝号；如果没有国号和帝号，则必须有年号。
    3. 月格式必须为1-12阿拉伯数字或“闰”/“閏”+数字1-12。
    4. 年和日必须为阿拉伯数字。
    """
    guo, di, era = rec['国号'], rec['帝号'], rec['年号']
    year, month, day = rec['年'], rec['月'], rec['日']
    # 规则4: 年/日数字检查
    if year and not year.isdigit():
        return '年必须为阿拉伯数字。'
    if day and not day.isdigit():
        return '日必须为阿拉伯数字。'
    # 规则1: 层级关系
    if day and not (month or year):
        return '输入了“日”，请同时输入“月”及“年”。'
    if month and not year:
        return '输入了“月”，请同时输入“年”。'
    # 规则2: 年号 vs 国号/帝号
    if not era:
      if not guo and not di:
        return '请填写年号或国号+帝号'
      else:
        if not guo or not di:
            return '未填写“年号”，请同时填写“国号”和“帝号”。'
    # 规则3: 月格式
    if month:
        if not re.match(r'^(?:闰|閏)?(?:[1-9]|1[0-2])$', month):
            return '月格式不正确，应为1-12数字或闰1-12。'
    return None

# 导出函数

def convert_and_download(df: pd.DataFrame, filename: str = "output_input.csv"):
    for col in ["国号", "帝号"]:
        if col not in df.columns:
            df[col] = ""
    df = df[["国号", "帝号", "年号", "年", "月", "日"]].fillna("")
    df = df.applymap(lambda x: cc.convert(str(x)) if isinstance(x, str) else x)
    df.to_csv(filename, index=False, header=False, encoding="utf-8")
    files.download(filename)
    print(f"✅ 下载完成：{filename}")

# Excel 上传分支

def upload_excel_ui():
    clear_output()
    display(widgets.HTML('<b>说明：</b>请上传含有“年号”、“年”、“月”、“日”列的 Excel 文件；系统会自动补齐“国号”和“帝号”。'))
    uploaded = files.upload()
    fname = list(uploaded.keys())[0]
    df = pd.read_excel(io.BytesIO(uploaded[fname]), dtype=str)
    print("✅ 上传并读取完成，前几行：")
    display(df.head())
    convert_and_download(df, f"{os.path.splitext(fname)[0]}_input.csv")

# 手动输入分支：单条录入

def manual_input_ui():
    clear_output()
    display(widgets.HTML('<b>说明：</b>请逐条输入各字段，然后点击“下一条”以添加，完成后点击“提交并处理”生成并下载 CSV。'))
    field_guo = widgets.Text(description='国号：')
    field_di = widgets.Text(description='帝号：')
    field_era = widgets.Text(description='年号：')
    field_year = widgets.Text(description='年：', placeholder='阿拉伯数字，例如5')
    field_month = widgets.Text(description='月：', placeholder='1-12或闰1-12')
    field_day = widgets.Text(description='日：', placeholder='阿拉伯数字，例如14')

    next_btn = widgets.Button(description='下一条')
    submit_btn = widgets.Button(description='提交并处理')
    manual_out = widgets.Output()

    def on_next(b):
        with manual_out:
            # 不清除输出，保留历史记录
            rec = {
                '国号': field_guo.value.strip(),
                '帝号': field_di.value.strip(),
                '年号': field_era.value.strip(),
                '年': field_year.value.strip(),
                '月': field_month.value.strip(),
                '日': field_day.value.strip()
            }
            err = validate_entry(rec)
            if err:
                print(f"❌ {err}")
                return
            entries.append(rec)
            label = f"✅ 已添加{rec['国号']}{rec['帝号']}{rec['年号']}{rec['年']}年{rec['月']}月{rec['日']}日"
            print(label)
            # 清空输入字段
            field_guo.value = ''
            field_di.value = ''
            field_era.value = ''
            field_year.value = ''
            field_month.value = ''
            field_day.value = ''

    def on_submit(b):
        with manual_out:
            manual_out.clear_output()
            if not entries:
                print("❌ 请先点击“下一条”添加至少一条记录。")
                return
            df = pd.DataFrame(entries)
            print("✅ 所有记录：")
            display(df)
            convert_and_download(df, 'manual_input.csv')

    next_btn.on_click(on_next)
    submit_btn.on_click(on_submit)

    display(
        widgets.VBox([
            widgets.HBox([field_guo, field_di]),
            widgets.HBox([field_era, field_year]),
            widgets.HBox([field_month, field_day]),
            widgets.HBox([next_btn, submit_btn]),
            manual_out
        ])
    )

# 主界面：模式选择 + 确定按钮
mode_selector = widgets.RadioButtons(
    options=[('上传 Excel 文件', 'excel'), ('手动输入/粘贴中历日期', 'manual')],
    description='选择模式：'
)
confirm_button = widgets.Button(description='确定')
ui_output = widgets.Output()

confirm_button.on_click(lambda b: (ui_output.clear_output(), upload_excel_ui() if mode_selector.value=='excel' else manual_input_ui()))

display(mode_selector, confirm_button, ui_output)


HTML(value='<b>说明：</b>请逐条输入各字段，然后点击“下一条”以添加，完成后点击“提交并处理”生成并下载 CSV。')